## Bank Churn Prediction Project

Businesses like banks which provide service have to worry about problem of 'Churn' i.e. customers
leaving and joining another service provider. It is important to understand which aspects of the service
influence a customer's decision in this regard. Management can concentrate efforts on improvement of
service, keeping in mind these priorities.


**Objective:**
Given a Bank customer, build a neural network based classifier that can determine whether they will leave
or not in the next 6 months.


Points Distribution:
The points distribution for this case is as follows:
1. Read the dataset
2. Drop the columns which are unique for all users like IDs (5 points)
3. Distinguish the features and target variable (5 points)
4. Divide the data set into training and test sets (5 points)
5. Normalize the train and test data (10 points)
6. Initialize & build the model. Identify the points of improvement and implement the same the same.(20)
7. Predict the results using 0.5 as a threshold (10 points)
8. Print the Accuracy score and confusion matrix (5 points)

### Dataset Description

The case study is from an open-source dataset from Kaggle.
The dataset contains 10,000 sample points with 14 distinct features such as CustomerId, CreditScore,
Geography, Gender, Age, Tenure, Balance etc.
Link to the Kaggle project site:
https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling

In [ ]:
!pip install tensorflow==2.0

     |████████████████████████████████| 86.3MB 46kB/s 
     |████████████████████████████████| 450kB 51.8MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 3.8MB 54.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=2748cc2a095533bff75e2b31573ff6ef5d34128f1750b446a6ce3c43456f7596
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully u

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, auc
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix


In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Read the dataset

In [ ]:
project_path = '/content/drive/My Drive/Colab Notebooks/'

In [ ]:
dataset_file = project_path + 'bank.csv'

In [ ]:
data = pd.read_csv(dataset_file)

In [ ]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
data.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

Drop the columns which are unique for all users like IDs (5 points)

In [ ]:
data['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [ ]:
#RowNumber #CustomerId and #Surname are unique hence dropping it
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


Distinguish the features and target variable (5 points)

In [ ]:
# Columns: Credit Score Geography	Gender	Age	Tenure	Balance	NumOfProducts	HasCrCard	IsActiveMember	EstimatedSalary
X = data.iloc[:,0:10].values 
# Column: Exited
y = data.iloc[:,10].values 

In [ ]:
# Encoding country categorical data (Object to  binary)
print(X[:8,1], 'is: ')

label_X_country_encoder = LabelEncoder()
X[:,1] = label_X_country_encoder.fit_transform(X[:,1])
print(X[:8,1])

['France' 'Spain' 'France' 'France' 'Spain' 'Spain' 'France' 'Germany'] is: 
[0 2 0 0 2 2 0 1]


In [ ]:
# Encoding gender
print(X[:6,2], 'is: ')

label_X_gender_encoder = LabelEncoder()
X[:,2] = label_X_gender_encoder.fit_transform(X[:,2])
print(X[:6,2])

['Female' 'Female' 'Female' 'Female' 'Female' 'Male'] is: 
[0 0 0 0 0 1]


In [ ]:
# Converting ordinal values to different dimensions 
countryhotencoder = ColumnTransformer([("countries", OneHotEncoder(), [1])], remainder="passthrough")

#X = countryhotencoder.fit_transform(X).toarray()
X = countryhotencoder.fit_transform(X)

In [ ]:
X.shape

(10000, 12)

In [ ]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [ ]:
# Remove Spain dimension
X = X[:,1:] 

Divide the data set into training and test sets (5 points)

In [ ]:
# Split the dataset into the train and test set

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

Normalize the train and test data (10 points)

In [ ]:
# Feature Scaling
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Initialize & build the model. Identify the points of improvement and implement the same the same.(20)

In [ ]:
# Initializing the ANN
classifier = Sequential()

In [ ]:
classifier.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='uniform'))

In [ ]:
# 1st hidden layer
classifier.add(Dense(6, activation='sigmoid', kernel_initializer='uniform'))

In [ ]:
# Adding the output layer
classifier.add(Dense(1, activation = 'sigmoid', kernel_initializer='uniform')) 

In [ ]:
# Optimize using the default learning rate and compile the model
classifier.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])

In [ ]:
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 6)                 72        
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [ ]:
classifier.fit(X_train, y_train,           
          validation_data=(X_test,y_test),
          epochs=100,
          batch_size=32)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.2075 - accuracy: 0.7960 - val_loss: 0.1824 - val_accuracy: 0.7975
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.1742 - accuracy: 0.7960 - val_loss: 0.1681 - val_accuracy: 0.7975
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.1664 - accuracy: 0.7960 - val_loss: 0.1640 - val_accuracy: 0.7975
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.1640 - accuracy: 0.7960 - val_loss: 0.1626 - val_accuracy: 0.7975
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.1630 - accuracy: 0.7960 - val_loss: 0.1620 - val_accuracy: 0.7975
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.1627 - accuracy: 0.7960 - val_loss: 0.1617 - val_accuracy: 0.7975
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.1625 - accuracy: 0.7960 - val_loss: 0.1616 - val_accuracy: 0.7975

Predict the results using 0.5 as a threshold (10 points)


In [ ]:
y_pred = classifier.predict(X_test)
print(y_pred)

[[0.20556968]
 [0.20624334]
 [0.20172228]
 ...
 [0.20138817]
 [0.20420489]
 [0.2055082 ]]


In [ ]:
# Convert the probability of a customer leaving the bank to boolean(true or false) using the cutoff value 0.5
y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


Print the Accuracy score and confusion matrix (5 points)

In [ ]:
cm1 = confusion_matrix(y_test, y_pred)
print(cm1)

[[1595    0]
 [ 405    0]]


In [ ]:
accuracy_model1 = ((cm1[0][0]+cm1[1][1])*100)/(cm1[0][0]+cm1[1][1]+cm1[0][1]+cm1[1][0])
print (accuracy_model1, '%')

79.75 %


Let me try to improve (optimise) the model using more parameters

In [ ]:
# Initializing the ANN
optimised_classifier = Sequential()
# This adds the input layer (by specifying input dimension) AND the first hidden layer (units)
optimised_classifier.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='uniform'))

In [ ]:
# Add hidden layer
optimised_classifier.add(Dense(activation = 'relu', units=6, kernel_initializer='uniform')) 

In [ ]:
# Add the output layer

optimised_classifier.add(Dense(activation = 'sigmoid', units=1, kernel_initializer='uniform')) 

In [ ]:
optimised_classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
optimised_classifier.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 6)                 72        
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 42        
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimised_classifier.fit(X_train, y_train,           
          validation_data=(X_test,y_test),
          epochs=100,
          batch_size=32)

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5633 - accuracy: 0.7954 - val_loss: 0.4409 - val_accuracy: 0.7975
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4378 - accuracy: 0.7960 - val_loss: 0.4305 - val_accuracy: 0.7975
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4303 - accuracy: 0.7960 - val_loss: 0.4249 - val_accuracy: 0.7975
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4259 - accuracy: 0.7960 - val_loss: 0.4202 - val_accuracy: 0.7975
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4216 - accuracy: 0.8048 - val_loss: 0.4146 - val_accuracy: 0.8260
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4169 - accuracy: 0.8251 - val_loss: 0.4095 - val_accuracy: 0.8350
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4129 - accuracy: 0.8311 - val_loss: 0.4060 - val_accuracy: 0.8405

2nd prediction of results using 0.5 as a threshold

In [ ]:
y_pred = optimised_classifier.predict(X_test)
print(y_pred)

[[0.20273343]
 [0.3420826 ]
 [0.1396132 ]
 ...
 [0.18157512]
 [0.10916205]
 [0.24121255]]


In [ ]:
# Convert the probabilities that a customer will leave the bank into bolean using 0.5 benchmark
y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


Accuracy score and confusion matrix

In [ ]:
cm2 = confusion_matrix(y_test, y_pred)
print(cm2)

[[1533   62]
 [ 212  193]]


In [ ]:
accuracy_model2 = ((cm2[0][0]+cm2[1][1])*100)/(cm2[0][0]+cm2[1][1]+cm2[0][1]+cm2[1][0])
print (accuracy_model2, '%')

86.3 %


Conclusion:
The improved (optimised) model has a better accuracy score